## 0. Summary
- 목적: 문서(JSONL)를 읽고 → 문서를 청크로 분할 → 임베딩 후 벡터DB(Chroma)에 저장 → 평가용 대화 로그에서 과학 상식 관련 여부 판별 → 독립 질의(standalone query) 생성 → 유사 문서 검색 → 답변 생성까지 파이프라인 구축
- 주요 구성:
  1) 데이터 로딩/스키마 변환
  2) 텍스트 청크 분할
  3) 임베딩 및 벡터스토어 구축(Chroma)
  4) 평가 대화 로딩 및 전처리
  5) 과학 상식 판별 체인
  6) Standalone Query 생성 체인
  7) Retriever로 관련 문서 검색
  8) 판별-검색-생성 통합 파이프라인


## 1. 문서 읽기
- documents.jsonl, eval.jsonl을 읽어 샘플 구조 확인 및 파이썬 dict 리스트로 적재
- LangChain Document로 변환하여 page_content와 metadata(doc_id, source, 길이 등) 구성
- 최대 문서 길이 등 간단한 통계 확인


In [1]:
import json
from pathlib import Path


# 데이터 디렉터리와 입력 파일 경로 설정
DIR_DATA = Path("../../data")
DOCUMENT_PATH = DIR_DATA / "documents.jsonl"
EVAL_PATH = DIR_DATA / "eval.jsonl"

In [2]:
def check_documents_file(filepath: Path):
    # JSONL 파일에서 첫 라인만 읽어 필드 구조(키 목록)와 예시를 확인
    with filepath.open() as f:
        for line in f:
            doc_json = json.loads(line)
            print("keys:", doc_json.keys())  # 필드 키 확인
            display(doc_json)  # 예시 한 건 출력
            break


# 샘플 프리뷰: 문서 파일과 평가 파일 각각 1건씩 확인
check_documents_file(DOCUMENT_PATH)
print("=" * 20)
check_documents_file(EVAL_PATH)

keys: dict_keys(['docid', 'src', 'content'])


{'docid': '42508ee0-c543-4338-878e-d98c6babee66',
 'src': 'ko_mmlu__nutrition__test',
 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.'}

keys: dict_keys(['eval_id', 'msg'])


{'eval_id': 78,
 'msg': [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]}

In [3]:
def load_from_json(filepath: Path) -> list[dict]:
    # JSONL 파일을 줄 단위로 읽어 파이썬 dict 리스트로 변환
    with filepath.open() as f:
        return [json.loads(line.strip()) for line in f]

In [4]:
from langchain_core.documents import Document


def load_documents(filepath: Path) -> list[Document]:
    """filepath 로 부터 jsonl 을 읽어서 Document list 만들기"""
    # raw dict를 LangChain Document로 변환
    documents = [
        Document(
            page_content=raw_document["content"],  # 본문
            metadata={
                "doc_id": raw_document["docid"],  # 문서 식별자
                "source": raw_document["src"],  # 출처
                "len": len(raw_document["content"]),  # 본문 길이(문자 수)
            },
        )
        for raw_document in load_from_json(filepath)
    ]
    return documents


# 문서 로딩 및 간단 출력
docs = load_documents(DOCUMENT_PATH)
print("total documents:", len(docs))
print(docs[0])

total documents: 4272
page_content='건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.' metadata={'doc_id': '42508ee0-c543-4338-878e-d98c6babee66', 'source': 'ko_mmlu__nutrition__test', 'len': 381}


In [5]:
# 로드된 문서들 중 가장 긴 본문 길이 확인
print("가장 긴 문서의 길이:", max([doc.metadata["len"] for doc in docs]))

가장 긴 문서의 길이: 1230


## 2. 문서 나누기
- 긴 문서를 검색 친화적으로 만들기 위해 고정 길이(chunk_size=512, overlap=256)로 분할
- 분할된 각 청크에 원본 metadata를 복사하고, 추가로 chunk_id, total_chunks, len(청크 길이) 부여
- 분할 이후 최대 청크 길이와 샘플 확인


In [6]:
from langchain_core.documents import Document  # 재확인(가독성)
from langchain_text_splitters import RecursiveCharacterTextSplitter


def split_document(documents: list[Document]) -> list[Document]:
    """문서를 적절히 chunking 하기"""
    total_documents = []
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,  # 각 청크 최대 길이
        chunk_overlap=256,  # 청크 간 겹침 길이(문맥 유지)
    )
    for document in documents:
        chunks = splitter.split_text(document.page_content)
        total_chunks = len(chunks)
        # 각 청크를 독립 Document로 변환(검색/RAG용)
        chunked_documents = [
            Document(
                page_content=chunk,
                metadata={
                    **document.metadata,
                    "len": len(chunk),  # 청크 길이
                    "chunk_id": idx,  # 청크 인덱스
                    "total_chunks": total_chunks,
                },
            )
            for idx, chunk in enumerate(chunks)
        ]
        total_documents.extend(chunked_documents)
    return total_documents


# 분할 실행 및 요약 출력
split_docs = split_document(docs)
print("total split documents:", len(split_docs))
print(split_docs[0])
print("가장 긴 문서의 길이:", max([doc.metadata["len"] for doc in split_docs]))

total split documents: 4480
page_content='건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.' metadata={'doc_id': '42508ee0-c543-4338-878e-d98c6babee66', 'source': 'ko_mmlu__nutrition__test', 'len': 381, 'chunk_id': 0, 'total_chunks': 1}
가장 긴 문서의 길이: 512


## 3. 문서를 Vector DB 에 저장하기
- 임베딩 백엔드 선택(예: Ollama/OpenAI/Upstage/HuggingFace) → 임베딩 객체 생성
- 벡터스토어(Chroma) 인스턴스화, 컬렉션 이름 지정
- 분할 문서를 임베딩하여 벡터스토어에 적재


In [7]:
from enum import Enum

from langchain_core.embeddings import Embeddings


class Platform(str, Enum):
    OLLAMA = "ollama"
    OPENAI = "openai"
    UPSTAGE = "upstage"
    HUGGINGFACE = "huggingface"


def get_embeddings(platform: Platform, model: str) -> Embeddings:
    """platform 별 Embeddings 반환"""
    # 선택한 플랫폼별 임베딩 클래스 로딩
    if platform == Platform.OLLAMA:
        from langchain_ollama import OllamaEmbeddings

        return OllamaEmbeddings(model=model)
    if platform == Platform.OPENAI:
        from langchain_openai import OpenAIEmbeddings

        return OpenAIEmbeddings(model=model)
    if platform == Platform.UPSTAGE:
        from langchain_upstage import UpstageEmbeddings

        return UpstageEmbeddings(model=model)
    if platform == Platform.HUGGINGFACE:
        from langchain_huggingface import HuggingFaceEmbeddings

        return HuggingFaceEmbeddings(model=model)
    raise ValueError(f"unknown platform: {platform}")


# 예시: Ollama 기반 bge 계열 임베딩 선택
ollama_embeddings = get_embeddings(Platform.OLLAMA, model="bge-large:335m")
ollama_embeddings

/Users/joyuiyeong/projects/upstageailab-ir-competition-ir-5/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OllamaEmbeddings(model='bge-large:335m', validate_model_on_init=False, base_url=None, client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [8]:
from langchain_chroma import Chroma
from langchain_core.vectorstores import VectorStore


class VectorStoreType(str, Enum):
    CHROMA = "chroma"


def make_chroma_vector_store(embeddings: Embeddings) -> Chroma:
    # 메모리(또는 기본 로컬) 기반 Chroma 인스턴스 생성
    return Chroma(
        embedding_function=embeddings,
        collection_name="science_common_knowledge",  # 컬렉션 이름
    )


def make_vector_store(vector_store_type: VectorStoreType, embeddings: Embeddings) -> VectorStore:
    if vector_store_type == VectorStoreType.CHROMA:
        return make_chroma_vector_store(embeddings)
    raise ValueError(f"unknown vector_store_type: {vector_store_type}")

In [9]:
# 벡터스토어 생성(Chroma) 및 객체 확인
vector_store = make_vector_store(VectorStoreType.CHROMA, ollama_embeddings)
vector_store

In [ ]:
# 분할 문서를 임베딩하여 벡터스토어에 적재
vector_store.add_documents(split_docs)

## 4. evaluation chat 에서 standalone query 를 찾기
- 평가용 대화 로그(eval.jsonl)를 LangChain 메시지 객체(HumanMessage/AIMessage) 시퀀스로 변환
- ChatOllama 등 LLM 인스턴스 준비
- 이후 단계에서 과학 상식 판별/Standalone Query 생성을 위해 대화 히스토리와 마지막 질문을 구조화


In [ ]:
from langchain_ollama import ChatOllama


# LLM 인스턴스(예: gpt-oss:20b) 초기화
llm = ChatOllama(model="gpt-oss:20b")

In [ ]:
from dataclasses import dataclass

from langchain_core.messages import AIMessage, HumanMessage


@dataclass(frozen=True)
class EvalDocument:
    eval_id: int
    chat_history: list[HumanMessage | AIMessage]  # 마지막 질문 이전까지의 히스토리
    question: str  # 마지막 turn의 사용자 질문


def convert_msg_to_message(msg: dict) -> HumanMessage | AIMessage:
    # eval.jsonl의 {role, content}를 LangChain 메시지로 매핑
    return HumanMessage(msg["content"]) if msg["role"] == "user" else AIMessage(msg["content"])


def load_eval_documents() -> list[EvalDocument]:
    # 평가용 대화 데이터 적재 및 구조화
    eval_documents = []
    for raw_document in load_from_json(EVAL_PATH):
        msg_list = raw_document["msg"]
        eval_document = EvalDocument(
            eval_id=raw_document["eval_id"],
            chat_history=[convert_msg_to_message(msg) for msg in msg_list[:-1]],  # 마지막 전까지
            question=msg_list[-1]["content"],  # 마지막 질문
        )
        eval_documents.append(eval_document)
    return eval_documents


# 로딩 및 개수 확인
eval_docs = load_eval_documents()
len(eval_docs)

### load 한 eval 확인
- 특정 인덱스의 EvalDocument 구조와 데이터 확인


In [ ]:
# 예시로 세 번째 항목 출력
eval_docs[2]

## 5. 과학 상식과 관련된 내용인지 아닌지 llm 으로 확인
- System 프롬프트로 역할/출력 형식 지정(Yes/No)
- 대화 히스토리 + 질문을 입력하여 과학 상식 관련성 판정 체인 구성
- 간단 샘플에 대해 체인 실행 결과 확인


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


# 과학 상식 판별용 시스템 프롬프트(Yes/No)
system_prompt = [
    "## Role",
    "당신은 질문이 과학 상식과 관련이 있는지 없는지를 판별하는 판별가입니다.",
    "## Instruction지금까지의 대화 내용을 보고 과학 상식에 대한 질문인지 답하세요.",
    "## Output Format",
    "- Yes: 과학 상식 질문일 때",
    "- No: 일반 대화 일 때",
    "## 지금까지 대화 내용",
]

# 템플릿: 히스토리 + 사용자 질문
determinator_template = ChatPromptTemplate.from_messages(
    [("system", system_prompt), MessagesPlaceholder("history"), ("user", "{question}")]
)
determinator_template

In [ ]:
# 템플릿이 올바르게 메시지를 구성하는지 간단 점검
determinator_template.invoke(
    {
        "history": eval_docs[0].chat_history,
        "question": eval_docs[0].question,
    }
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser


# 체인 구성: 프롬프트 → LLM → 문자열 파서
determinator_chain = determinator_template | llm | StrOutputParser()

# 샘플 실행
determinator_chain.invoke(
    {
        "history": eval_docs[0].chat_history,
        "question": eval_docs[0].question,
    }
)

In [17]:
# 다른 샘플에 대해서도 판별 결과 확인
print("대화 내용:")
print("    ", eval_docs[2].chat_history)
print("질문:", eval_docs[2].question)
print("과학 상식과 관련된 내용인가?")
determinator_chain.invoke(
    {
        "history": eval_docs[2].chat_history,
        "question": eval_docs[2].question,
    }
)

대화 내용:
     [HumanMessage(content='기억 상실증 걸리면 너무 무섭겠다.', additional_kwargs={}, response_metadata={}), AIMessage(content='네 맞습니다.', additional_kwargs={}, response_metadata={})]
질문: 어떤 원인 때문에 발생하는지 궁금해.
과학 상식과 관련된 내용인가?


'Yes'

In [18]:
print("대화 내용:")
print("    ", eval_docs[3].chat_history)
print("질문:", eval_docs[3].question)
print("과학 상식과 관련된 내용인가?")
determinator_chain.invoke(
    {
        "history": eval_docs[3].chat_history,
        "question": eval_docs[3].question,
    }
)

대화 내용:
     []
질문: 통학 버스의 가치에 대해 말해줘.
과학 상식과 관련된 내용인가?


'No'

## 6. Standalone Query 뽑기
- 대화 맥락을 반영해 검색 최적화된 독립 질의로 재작성
- 규칙: 맥락 보완, 핵심어 포함, 검색 친화 키워드 포함
- 체인: 프롬프트 → LLM → 문자열 파서


In [19]:
standalone_q_system = [
    "## Role",
    "당신은 최적의 검색 키워드 생성기 입니다.",
    "## Instruction",
    "지금까지의 대화 내용을 바탕으로, 문서 검색에 적합한 독립적인 쿼리를 생성해주세요.",
    "## Generation Rule",
    "1. 대화 맥락을 참고하여 완전한 질문으로 변환",
    "3. 핵심 단어를 포함하여 변환",
    "2. 검색에 최적화된 키워드 포함",
    "## Example",
    "입력: '피임약 괜찮아?",
    "출력: '피임약의 정의, 효과, 부작용 및 사용법에 대해서 알려줘",
]
standalone_query_prompt = ChatPromptTemplate.from_messages(
    [("system", standalone_q_system), MessagesPlaceholder("history"), ("user", "{question}")]
)
standalone_query_prompt

ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [20]:
# 템플릿 메시지 구성 점검
standalone_query_prompt.invoke(
    {
        "history": eval_docs[0].chat_history,
        "question": eval_docs[0].question,
    }
)

ChatPromptValue(messages=[SystemMessage(content=[{'type': 'text', 'text': '## Role'}, {'type': 'text', 'text': '당신은 최적의 검색 키워드 생성기 입니다.'}, {'type': 'text', 'text': '## Instruction'}, {'type': 'text', 'text': '지금까지의 대화 내용을 바탕으로, 문서 검색에 적합한 독립적인 쿼리를 생성해주세요.'}, {'type': 'text', 'text': '## Generation Rule'}, {'type': 'text', 'text': '1. 대화 맥락을 참고하여 완전한 질문으로 변환'}, {'type': 'text', 'text': '3. 핵심 단어를 포함하여 변환'}, {'type': 'text', 'text': '2. 검색에 최적화된 키워드 포함'}, {'type': 'text', 'text': '## Example'}, {'type': 'text', 'text': "입력: '피임약 괜찮아?"}, {'type': 'text', 'text': "출력: '피임약의 정의, 효과, 부작용 및 사용법에 대해서 알려줘"}], additional_kwargs={}, response_metadata={}), HumanMessage(content='나무의 분류에 대해 조사해 보기 위한 방법은?', additional_kwargs={}, response_metadata={})])

In [21]:
# 체인 구성 및 실행
standalone_query_chain = standalone_query_prompt | llm | StrOutputParser()
query = standalone_query_chain.invoke(
    {
        "history": eval_docs[0].chat_history,
        "question": eval_docs[0].question,
    }
)
query = query.strip()
query  # 생성된 독립 질의

'나무 분류 조사를 위한 단계별 방법과 필요한 자료는 무엇인가?'

## 7. Retriever 로 연관 문서 찾기
- 생성한 standalone query로 벡터스토어에서 유사 문서 상위 k개 검색
- relevance score와 함께 결과 확인


In [22]:
top_k = 3
results = vector_store.similarity_search_with_relevance_scores(query, k=top_k)
len(results)  # 검색된 문서 수

3

In [23]:
# 1위 결과(문서와 스코어 튜플)
print(results[0])

(Document(id='feedf569-ac11-40d5-ae95-28cbe6d08579', metadata={'len': 512, 'doc_id': '25eee08d-e182-49a1-a921-8409df136ffb', 'chunk_id': 0, 'source': 'ko_ai2_arc__ARC_Challenge__test', 'total_chunks': 2}, page_content='노를 앞으로 젓는 데는 물과 바람의 마찰을 극복할 힘이 필요합니다. 배에 대항하는 힘을 극복하는 데 가장 도움이 되는 요소는 무엇인가요?\n\n배 안에서 노를 젓는 사람들의 수가 가장 도움이 되는 요소입니다. 배 안에서 노를 젓는 사람들은 물과 바람의 저항을 극복하기 위해 힘을 발휘합니다. 그들의 힘과 협력은 배를 움직이는 데 큰 영향을 미칩니다. 노를 젓는 사람들이 많을수록 더 많은 힘을 발휘할 수 있고, 따라서 물과 바람의 마찰을 극복하는 데 더 도움이 됩니다.\n\n또한, 노를 젓는 사람들의 기술과 경험도 중요한 요소입니다. 노를 젓는 사람들은 물과 바람의 조건을 파악하고, 적절한 기술과 전략을 사용하여 효율적으로 노를 젓습니다. 그들의 경험과 노하우는 배를 움직이는 데 큰 도움이 됩니다.\n\n마지막으로, 배 안에서 노를 젓는 사람들의 의지와 열정도 중요합니다. 노를 젓는 일은 힘들고 힘든 상황에서도 계속해서 노력해야 합니다. 의지와 열정이 강한 사람들은 어려움을 극복하고 목표를 달성하는 데 도움이 됩니다.'), 0.8297658653648305)


In [24]:
# 2위 결과
print(results[1])

(Document(id='f13ea54a-0057-41e1-898d-fb03e950b039', metadata={'chunk_id': 0, 'doc_id': '4106101f-fd00-4815-abb6-50a1a5b4c201', 'source': 'ko_mmlu__conceptual_physics__validation', 'total_chunks': 1, 'len': 366}, page_content='대기압은 대기의 무게로 인해 발생합니다. 대기는 지구 주변에 존재하는 공기의 질량입니다. 지구의 중력에 의해 대기는 지구 표면을 둘러싸고 있으며, 이 대기의 무게가 대기압을 형성합니다. 대기압은 지구 표면에서의 공기의 무게로 인해 발생하며, 이는 지구의 대기권에서 높이가 올라갈수록 감소합니다. 대기압은 대기의 무게에 의해 발생하는데, 이는 대기 분자들이 지구 표면을 누르는 힘으로 작용합니다. 이러한 대기압은 우리 주변에서 매우 중요한 역할을 합니다. 대기압은 날씨, 기후, 바람, 기상 조건 등을 결정하는데 영향을 미치며, 또한 생명체들에게 산소를 공급하는 역할도 합니다. 따라서 대기압은 지구 생태계와 인간 생활에 매우 중요한 역할을 하는 것입니다.'), 0.8270985915201065)


In [25]:
# 3위 결과
print(results[2])

(Document(id='667a0efa-a314-4223-807e-e5cb4f21a911', metadata={'len': 339, 'source': 'ko_mmlu__high_school_biology__test', 'total_chunks': 1, 'chunk_id': 0, 'doc_id': 'af53bfa4-6d38-4d61-b549-d0388f9b6948'}, page_content='중립 변이가 정말로 "중립"이라면, 그것은 다음에 영향을 미치지 않아야 합니다. 이는 상대적 적합도에 의해 결정됩니다. 상대적 적합도는 개체의 유전자 변이가 생존과 번식에 얼마나 영향을 미치는지를 나타내는 척도입니다. 중립 변이는 개체의 생존 능력이나 번식 능력에 영향을 주지 않으며, 따라서 자연 선택에 의해 제거되지 않습니다. 이러한 중립 변이는 종의 유전적 다양성을 유지하는 데 중요한 역할을 합니다. 중립 변이는 종의 진화와 적응력을 유지하는 데 기여하며, 종 내에서의 유전적 다양성은 생존과 번식에 필수적입니다. 따라서 중립 변이는 종의 생존과 번식에 영향을 미치지 않는 중요한 유전적 요소입니다.'), 0.8179424310254937)


## 8. 과학 질문 판별기와 검색 쿼리 생성기 통합하기
- 파이프라인:
  1) 과학 상식 여부 판정(Yes/No)
  2) Yes라면 Standalone Query 생성 → 유사 문서 검색 → 관련 정보를 system 입력으로 답변 생성
  3) No라면 판별용 프롬프트 기반으로 경량 답변
- 제출 요소: eval_id, standalone_query, answer, topk(문서 id 목록), references(문서/스코어)


In [26]:
def make_submission_element(eval_document: EvalDocument):
    # 출력 스키마 기본값
    element = {
        "eval_id": eval_document.eval_id,
    }
    # 공통 입력 파라미터
    params = {
        "history": eval_document.chat_history,
        "question": eval_document.question,
    }
    # 1) 과학 상식 여부 판정
    answer_with_docs = determinator_chain.invoke(params)
    if answer_with_docs == "Yes":
        # 2) Standalone Query 생성
        standalone_query = standalone_query_chain.invoke(params)
        # 3) 관련 문서 검색
        related_docs = vector_store.similarity_search_with_relevance_scores(standalone_query, k=top_k)
        # 4) 관련 정보 주입하여 답변 생성
        prompt = [
            "## Role",
            "당신의 최고의 과학 전문가입니다.",
            "## Instruction",
            "주어진 추가 정보를 바탕으로 사용자의 질문에 알맞은 답을 해주세요.",
            "## Additional Information",
            "{information}## 사용자 질문",
        ]
        template = ChatPromptTemplate.from_messages([("system", prompt), ("user", "{question}")])
        answer = (template | llm).invoke(
            {"information": "\n\n".join(doc.page_content for doc, _ in related_docs), "question": standalone_query}
        )
        # 제출 요소 구성
        element["standalone_query"] = standalone_query
        element["answer"] = answer
        element["topk"] = [doc.metadata["doc_id"] for doc, _ in related_docs]
        element["references"] = [{"score": score, "content": doc.page_content} for doc, score in related_docs]
    else:
        # 과학 상식이 아닐 경우: 히스토리를 사용한 간단 응답
        template = ChatPromptTemplate.from_messages(
            [("system", system_prompt), MessagesPlaceholder("history"), ("user", "{question}")]
        )
        answer = (template | llm).invoke(params)
        element["standalone_query"] = params["question"]
        element["answer"] = answer
    return element


# 파이프라인 단건 테스트
make_submission_element(eval_docs[0])

{'eval_id': 78,
 'standalone_query': '나무 분류를 조사하기 위한 구체적인 방법과 절차는 어떻게 되나요?',
 'answer': AIMessage(content='## 나무 분류(식물분류학적 조사)를 위한 단계별 실전 가이드  \n\n아래 절차는 현장 조사(필드) → 실험실 분석 → 데이터베이스 구축 → 최종 분류 결론까지 한 번에 따라가면, 과학적으로 신뢰할 수 있는 나무 분류 결과를 얻을 수 있는 **표준화된 흐름**입니다.  \n필요한 장비·소프트웨어·지식 수준은 “초급 → 중급 → 고급” 단계별로 구분해 두었습니다.\n\n---\n\n### 1. 사전 준비 (Pre‑field)\n\n| 단계 | 주요 내용 | 준비물/도구 | 주의 사항 |\n|------|----------|-------------|----------|\n| 1‑1 | **목표 설정** (예: 특정 지역, 특정 종군, 혹은 전면 조사) | 조사 설계서(연구 목적, 가설, 일정) | 연구 목적에 따라 표본 수가 달라집니다. |\n| 1‑2 | **법적·윤리적 승인** (지자체 허가, 수목 보호구역 등) | 허가서, 수집 동의서 | 수집이 제한된 지역은 반드시 허가를 받아야 함. |\n| 1‑3 | **자료 수집 방법론** (모집단, 표본 크기, 표본 위치 선정) | 정규화된 표본 수(예: 30개) | 무작위 추출 또는 층화 추출을 명확히 기록. |\n| 1‑4 | **장비 체크리스트** | GPS, 전자 저울, 측정 테이프, 칼, 수집용 가방, 라벨(프린터) | 장비가 정상 동작 여부를 사전 테스트. |\n| 1‑5 | **분류 키 및 참고 문헌 준비** | 지역 Flora, 식물 분류 책, 온라인 데이터베이스 | 최신 버전과 과거 문헌을 병행 활용. |\n\n> **Tip**: 지역별로 수집이 제한된 종(예: 희귀종, 보호종)은 반드시 생태조사 허가와 함께 “필수 최소 수집” 원칙을 적용합니다.\n\n---\n\n### 2. 현장 조사 (Field Work)